### doc_topics
ESTABELECE RELAÇÕES ENTRE DOCUMENTOS E TÓPICOS NA BASE DE DADOS NO MYSQL. TRABALHAMOS COM O ACERVO **ANTONIO AZEREDO DA SILVEIRA, MINISTÉRIO DAS RELAÇÕES EXTERIORES**.

### Observações

#### Sobre como cria tabela CSV com limite documentos por tópico

### Código

In [1]:
import nltk
import os
import codecs
import string
import matplotlib.pyplot as plt
import matplotlib as mpl
from gensim import corpora, models, similarities #Latent Dirichlet Allocation implementation with Gensim
import pyLDAvis
import pyLDAvis.gensim
from IPython.display import clear_output

import getpass
from sshtunnel import SSHTunnelForwarder
import pymysql

C:\Anaconda3\lib\site-packages\funcy\decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
C:\Anaconda3\lib\site-packages\funcy\decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
C:\Anaconda3\lib\site-packages\funcy\decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
C:\Anaconda3\lib\site-packages\funcy\decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
C:\Anaconda3\lib\site-packages\funcy\decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
C:\Anaconda3\lib\site-packages\funcy\decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use 

In [2]:
outputs = "../outputs/"

In [4]:
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
Seleciona textos da base mysql para fazer modelagem de tópicos
NOTA: DEIXEI EM RAW PORQUE ESSA EXTRAÇÃO TORNA O CÓDIGO MUITO LENTO, MAS DEVE SER FEITO DESSA FORMA NO CPU DO RENATO
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
count = 0
texts = []
pass_sshkey = getpass.getpass()
pass_mysql = getpass.getpass()

with SSHTunnelForwarder(('200.20.164.147', 22),
                        ssh_private_key = "C:/Users/marcelo.ribeiro/Dropbox/A-Marcelo/Educação-Trabalho/2016-CPDOC/Remoto/marcelo_priv_rsa",
                        #ssh_private_key = '/home/rsouza/.ssh/id_rsa',
                        ssh_private_key_password = pass_sshkey,
                        ssh_username="marcelobribeiro",
                        #ssh_username="rsouza",
                        remote_bind_address=('127.0.0.1', 3306)) as server:
    
    conn = pymysql.connect(host='localhost', 
                           port=server.local_bind_port, 
                           user='marcelobribeiro', 
                           #user='rsouza',
                           passwd=pass_mysql,
                           db='CPDOC_AS',
                           use_unicode=True, 
                           charset="utf8")
    cur = conn.cursor()
    
    '''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
    captura documentos da tabela docs para posterior modelagem de tópicos
    '''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
    cur.execute("SELECT * FROM CPDOC_AS.docs")
    data = cur.fetchall()
    numrows = cur.rowcount
    percentil = numrows/100
    
    for row in data:
        count += 1
        if row is None: break

        if count % percentil == 0: 
            clear_output()
            print(int(count/percentil),'% done')
        
        text =  row[1]  
        text = text.split()
        symbols = [x for x in string.punctuation]
        text = [p for p in text if p not in symbols]
        text = [p.strip(string.punctuation) for p in text]
        text = [p for p in text if not p.isdigit()]
        text = [p for p in text if len(p)>1]
        texts.append(text)
        
        #if count == 3: break #amostra de apenas 3 linhas. No computador do Renato, não usar o break.

········
········


In [ ]:
# load nltk's English stopwords as variable called 'stopwords'

additional_words = ['mr','one', 'two', 'three', 'four', 
                    'five', 'um', 'dois', 'três', 'quatro', 
                    'cinco', 'janeiro', 'fevereiro', 'março', 
                    'abril', 'maio', 'junho', 'julho', 'agosto', 
                    'setembro', 'outubro', 'novembro', 'dezembro', 
                    'january', 'february', 'march', 'april', 'may', 
                    'june', 'july', 'august', 'september', 
                    'october', 'november', 'december', 'países', 
                    'ser', 'país', 'ainda', 'milhões', 'maior', 
                    'anos', 'grande', 'apenas', 'outros', 'pode', 
                    'parte', 'partes', 'item', 'vossa', 'nota', 
                    'havia', 'pt', 'vg', 'ptvg', 'eh', 'nr', 'hrs', 
                    'pais', 'parte', 'hoje', 'brasemb', 'ontem', 
                    'dia', 'countries', 'would', 'new', 'also', 
                    'must', 'draft', 'shall', 'item', 'page', 
                    'th', 'anos', 'ii', 'dias', 'poderá', 'caso', 
                    'casos', 'qualquer', 'ano', 'mil', 'pessoas', 
                    'único', 'única', 'únicos', 'únicas', 'índice', 
                    'expedido', 'co', 'mm', 'er', 'via', 'ww', 'ra', 
                    'ia', 'ca', 'nu', 'wa', 'aa', 'ms', 'dc', 'mmm', 'pa']

stopwords = nltk.corpus.stopwords.words('english') + \
            nltk.corpus.stopwords.words('portuguese') + \
            nltk.corpus.stopwords.words('french') + \
            nltk.corpus.stopwords.words('spanish') + \
            additional_words
            
stopwords = list(set(stopwords))

In [5]:
#%time texts = [[word for word in text] for text in texts]
%time texts = [[word for word in text if word not in stopwords] for text in textos]

Wall time: 0 ns


In [6]:
print(len(texts[0]))

218


In [7]:
dictionary = corpora.Dictionary(texts)
dictionary.filter_extremes(no_below=1, no_above=0.8)
dictionary.filter_tokens(bad_ids=[0,]) #retira palavras a partir do id
corpus = [dictionary.doc2bow(text) for text in texts]

In [8]:
len(corpus)

3

In [9]:
%time lda = models.LdaModel(corpus, num_topics=50, id2word=dictionary, passes=10)

Wall time: 498 ms


In [10]:
lda.print_topics(-1, num_words=20)

[(0,
  '0.004*retransmissão + 0.004*vaqui + 0.004*nio + 0.004*el + 0.004*sponsors + 0.004*aún + 0.004*sabes + 0.004*excelência + 0.004*ix + 0.004*seja + 0.004*diciembre + 0.004*bem + 0.004*ainda + 0.004*espanha + 0.004*credenciales + 0.004*as + 0.004*relaciones + 0.004*original + 0.004*communic + 0.004*retransmito'),
 (1,
  '0.004*retransmissão + 0.004*vaqui + 0.004*nio + 0.004*el + 0.004*sponsors + 0.004*aún + 0.004*sabes + 0.004*excelência + 0.004*ix + 0.004*seja + 0.004*diciembre + 0.004*bem + 0.004*ainda + 0.004*espanha + 0.004*credenciales + 0.004*as + 0.004*relaciones + 0.004*original + 0.004*communic + 0.004*retransmito'),
 (2,
  '0.005*the + 0.005*of + 0.004*nations + 0.004*united + 0.004*draft + 0.004*and + 0.004*resolution + 0.004*go:-au + 0.004*session + 0.004*peru + 0.004*eat + 0.004*oman + 0.004*chile + 0.004*agenda + 0.004*republic + 0.004*jamaica + 0.004*distr + 0.004*madagascar + 0.004*salvador + 0.004*on'),
 (3,
  '0.004*retransmissão + 0.004*vaqui + 0.004*nio + 0.004*

In [11]:
lda.show_topics(num_topics=-1)

[(0,
  '0.004*diciembre + 0.004*ainda + 0.004*sabes + 0.004*excelência + 0.004*ix + 0.004*seja + 0.004*retransmissão + 0.004*bem + 0.004*relaciones + 0.004*credenciales'),
 (1,
  '0.004*diciembre + 0.004*ainda + 0.004*sabes + 0.004*excelência + 0.004*ix + 0.004*seja + 0.004*retransmissão + 0.004*bem + 0.004*relaciones + 0.004*credenciales'),
 (2,
  '0.005*the + 0.005*of + 0.004*nations + 0.004*united + 0.004*draft + 0.004*and + 0.004*resolution + 0.004*go:-au + 0.004*session + 0.004*peru'),
 (3,
  '0.004*diciembre + 0.004*ainda + 0.004*sabes + 0.004*excelência + 0.004*ix + 0.004*seja + 0.004*retransmissão + 0.004*bem + 0.004*relaciones + 0.004*credenciales'),
 (4,
  '0.004*diciembre + 0.004*ainda + 0.004*sabes + 0.004*excelência + 0.004*ix + 0.004*seja + 0.004*retransmissão + 0.004*bem + 0.004*relaciones + 0.004*credenciales'),
 (5,
  '0.004*diciembre + 0.004*ainda + 0.004*sabes + 0.004*excelência + 0.004*ix + 0.004*seja + 0.004*retransmissão + 0.004*bem + 0.004*relaciones + 0.004*cred

In [12]:
topics_matrix = lda.show_topics(formatted=False, num_words=20, num_topics=-1)
topics_matrix[11]

(11,
 [('retransmissão', 0.0040983606557377051),
  ('vaqui', 0.0040983606557377051),
  ('nio', 0.0040983606557377051),
  ('el', 0.0040983606557377051),
  ('sponsors', 0.0040983606557377051),
  ('aún', 0.0040983606557377051),
  ('sabes', 0.0040983606557377051),
  ('excelência', 0.0040983606557377051),
  ('ix', 0.0040983606557377051),
  ('seja', 0.0040983606557377051),
  ('diciembre', 0.0040983606557377051),
  ('bem', 0.0040983606557377051),
  ('ainda', 0.0040983606557377051),
  ('espanha', 0.0040983606557377051),
  ('credenciales', 0.0040983606557377051),
  ('as', 0.0040983606557377051),
  ('relaciones', 0.0040983606557377051),
  ('original', 0.0040983606557377051),
  ('communic', 0.0040983606557377051),
  ('retransmito', 0.0040983606557377051)])

#### Gera a distribuição de tópicos para um novo documento
Alimenta a base mysql com dados de score (relação) entre tópicos e documentos

In [13]:
count = 0
#percentil = int(len(cursor.rowcount/100)

pass_sshkey = getpass.getpass()
pass_mysql = getpass.getpass()
with SSHTunnelForwarder(('200.20.164.147', 22),
                        ssh_private_key = "C:/Users/marcelo.ribeiro/Dropbox/A-Marcelo/Educação-Trabalho/2016-CPDOC/Remoto/marcelo_priv_rsa",
                        #ssh_private_key = '/home/rsouza/.ssh/id_rsa',
                        ssh_private_key_password = pass_sshkey,
                        ssh_username="marcelobribeiro",
                        #ssh_username="rsouza",
                        remote_bind_address=('127.0.0.1', 3306)) as server:
    
    conn = pymysql.connect(host='localhost', 
                           port=server.local_bind_port, 
                           user='marcelobribeiro', 
                           #user='rsouza',
                           passwd=pass_mysql,
                           db='CPDOC_AS',
                           use_unicode=True, 
                           charset="utf8")
    cur = conn.cursor()
    '''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
    insere dados na base mysql
    captura documentos da tabela docs, transfere para a tabela topic-doc e insere dados de tópicos
    '''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
    
    cur.execute("DROP TABLE IF EXISTS topic_doc")
    cur.execute('''CREATE TABLE IF NOT EXISTS topic_doc 
               (doc_id VARCHAR(31), topic_id smallint(6), topic_score FLOAT
               DEFAULT NULL)
               ENGINE=MyISAM DEFAULT CHARSET='utf8';''')

    cur.execute("SELECT id,body FROM CPDOC_AS.docs")
    data = cur.fetchall()
    for row in data:
        count += 1 # for break event
        if row is None: break
        
        text =  row[1]
        text = text.split()
        text_bow = dictionary.doc2bow(text)
        print(lda[text_bow])
        score_list = lda[text_bow]
        doc_id = row[0]
        for score in score_list:
            topic_id = str(score[0])
            topic_score = str(score[1])
            query = "INSERT INTO topic_doc VALUES (%s, %s, %s)"
            cur.execute(query, (doc_id, topic_id, topic_score))
        print(doc_id, topic_id, topic_score)   
        cur.execute("ALTER TABLE CPDOC_AS.topic_doc ORDER BY topic_id ASC, topic_score DESC")
        
        #if count == 10: break #amostra de apenas 10 linhas

········
········
[(26, 0.99467391304347696)]
rb_1974.04.17_doc_I-66 26 0.994673913043
[(49, 0.97958333333333325)]
onu_1974.03.12_doc_II-49 49 0.979583333333
[(34, 0.97822222222222122)]
onu_1974.03.12_doc_V-2 34 0.978222222222
[(26, 0.73621051069686505), (49, 0.2464443565597729)]
be_1974.04.16_doc_VI-18 49 0.246421600596
[(26, 0.81898279059754375), (49, 0.16958863797388582)]
be_1977.06.01_doc_II-21 49 0.169449675672
[(26, 0.48448725331096754), (34, 0.052752502340245699), (49, 0.45967827713567128)]
ag_1974.03.13_doc_III-7 49 0.459591147144
[(26, 0.12210141495085997), (49, 0.83789858504913994)]
ag_1973.11.20_doc_VI-75 49 0.837979085477
[(26, 0.79448591007198721), (49, 0.11824136265528624)]
d_1974.04.23_doc_XXXII-8 49 0.118821829911
[(26, 0.78184023928953517), (49, 0.21375526632844322)]
d_1974.03.26_doc_XXXII-32 49 0.213765446739
[(26, 0.80873035621816924), (34, 0.013018602669232748), (49, 0.17781464371241248)]
d_1974.03.26_doc_XXII-9 49 0.177818561781


#### Visualizando os tópicos com o PyLDAvis

In [ ]:
# https://cran.r-project.org/web/packages/LDAvis/vignettes/details.pdf
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda, corpus, dictionary)

In [ ]:
data_ldavis = pyLDAvis.gensim.prepare(lda, corpus, dictionary)
pyLDAvis.save_html(data_ldavis, os.path.join(outputs,'pyldavis_output.html'))